In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
#import outcome file with org_uuid
main_df = pd.read_csv(R"d:/msc-project/data/final/outcome_final_v1.csv",encoding='utf-8',
                         parse_dates= ['founded_on','seed_date','series_a_date','series_b_date','series_c_date'])
main_processing_df = main_df[['org_uuid']]

In [3]:
main_df.head(3)

,org_uuid,name,status,founded_on,last_funding_on,num_funding_rounds,seed_date,series_a_date,first_fund_date,series_b_date,...,closed_on,founded_to_first_fund_years,first_fund_to_series_a_years,first_fund_to_series_b_years,first_fund_to_series_c_years,first_fund_to_acquired_years,first_fund_to_ipo_years,first_fund_to_close_years,last_fund_to_end_date_years,outcome
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,Airgo Networks,acquired,2000-05-01,2005-04-01,6.0,NaT,2000-12-01,2000-12-01,2002-02-19,...,NaN,0.6,0.0,1.22,2.41,6.0,NaN,NaN,15.76,1
1,9c8adac6-5c8e-9344-b763-6beab966c63c,Nauticus Networks,acquired,2000-10-01,2002-06-12,2.0,NaT,2001-02-12,2001-02-12,2002-06-12,...,NaN,0.4,0.0,1.33,NaN,2.9,NaN,NaN,18.57,1
2,6749cc07-efed-ff09-4efe-43faf6f580de,BioProcessors,acquired,2000-10-01,2007-08-06,5.0,NaT,2002-02-01,2002-02-01,2003-07-11,...,NaN,1.3,0.0,1.44,4.36,7.1,NaN,NaN,13.41,1


In [4]:
len(main_processing_df)

28727

In [5]:
#import main organisation csv
org_ori_df = pd.read_csv(R"d:/msc-project/data/pre-processed/organisations_preprocessed.csv",encoding='utf-8',
                         parse_dates= ['founded_on','went_public_on','acquired_on','closed_on'])

In [6]:
org_ori_df.head(3)

,org_uuid,name,legal_name,homepage_url,country_code,state_code,region,city,address,postal_code,...,acquirer_state_code,acquirer_region,acquirer_city,acquisition_type,acquired_on,price_usd,price,price_currency_code,parent_uuid,parent_name
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,...,NY,New York,New York,acquisition,2013-12-16,30000000.0,30000000.0,USD,NaN,NaN
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,"Digg Holdings, LLC",http://www.digg.com,USA,NY,New York,New York,NaN,NaN,...,NY,New York,New York,acquisition,2012-07-12,16000000.0,16000000.0,USD,NaN,NaN
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"Facebook, Inc.",http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [7]:
org_ori_df.columns

Index(['org_uuid', 'name', 'legal_name', 'homepage_url', 'country_code',
       'state_code', 'region', 'city', 'address', 'postal_code', 'status',
       'short_description', 'category_list', 'category_groups_list',
       'num_funding_rounds', 'total_funding_usd', 'total_funding',
       'total_funding_currency_code', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'primary_role', 'num_exits',
       'description', 'ipo_uuid', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'acquisition_uuid', 'acquirer_uuid',
       'acquirer_name', 'acquirer_country_code', 'acquirer_state_code',
       'acquirer_region', 'acquirer_city', 'acquisition_type

In [8]:
org_processing_df = org_ori_df[['org_uuid','category_groups_list']]

In [9]:
org_processing_df.head(3)

,org_uuid,category_groups_list
0,e1393508-30ea-8a36-3f96-dd3226033abd,"Content and Publishing,Internet Services,Media..."
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,"Internet Services,Media and Entertainment"
2,df662812-7f97-0b43-9d3e-12f64f504fbb,"Apps,Community and Lifestyle,Content and Publi..."


In [10]:
#create a list of unique categories
#categories_list = list(org_processing['category_list'])
#unique_categories = set()

#for category_list in categories_list:
    #category_list = category_list.split(',')
    #for category in category_list:
        #unique_categories.add(category)
        
#unique_categories = list (unique_categories)
#unique_categories.sort()

In [11]:
#assign one hot encode for category_list for each company
#for category in unique_categories:
    #column_name = 'is_'+category.lower().replace(' ','_')
    #one_hot = list()

    #for category_list in categories_list:
        #category_list = category_list.split(',')
        #if category in category_list:
            #one_hot.append(1)
        #else:
            #one_hot.append(0)

    #org_processing[column_name]=one_hot

In [12]:
org_processing_df_v1 = pd.merge(main_processing_df,
                               org_processing_df,
                               on='org_uuid',
                               how='left')
org_processing_df_v1.head(3)

,org_uuid,category_groups_list
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,"Hardware,Mobile,Software"
1,9c8adac6-5c8e-9344-b763-6beab966c63c,"Information Technology,Other,Privacy and Security"
2,6749cc07-efed-ff09-4efe-43faf6f580de,"Biotechnology,Health Care,Science and Engineering"


In [13]:
#create a list of unique category groups
groups_list = list(org_processing_df_v1['category_groups_list'])
unique_groups = set()

for group_list in groups_list:
    group_list = group_list.split(',')
    for group in group_list:
        unique_groups.add(group)
        
unique_groups = list (unique_groups)
unique_groups.sort()

In [14]:
len(unique_groups)

47

In [15]:
#assign one hot encode for category_list for each company
for group in unique_groups:
    column_name = 'is_'+group.lower().replace(' ','_')
    one_hot = list()

    for group_list in groups_list:
        group_list = group_list.split(',')
        if group in group_list:
            one_hot.append(1)
        else:
            one_hot.append(0)

    org_processing_df_v1[column_name]=one_hot
org_processing_df_v1.head(3)

,org_uuid,category_groups_list,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,...,is_professional_services,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,"Hardware,Mobile,Software",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,9c8adac6-5c8e-9344-b763-6beab966c63c,"Information Technology,Other,Privacy and Security",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6749cc07-efed-ff09-4efe-43faf6f580de,"Biotechnology,Health Care,Science and Engineering",0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [16]:
org_processing_df_v1.drop(columns=['category_groups_list'],inplace=True)

In [17]:
org_processing_df_v1.head(3)

,org_uuid,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,...,is_professional_services,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,9c8adac6-5c8e-9344-b763-6beab966c63c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6749cc07-efed-ff09-4efe-43faf6f580de,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [18]:
#one hot encode for country
org_processing_df_v2 = pd.merge(main_processing_df,
                                org_ori_df[['org_uuid','country_code']],
                                on='org_uuid',
                                how='left')

In [19]:
org_processing_df_v2.head(2)

,org_uuid,country_code
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,USA
1,9c8adac6-5c8e-9344-b763-6beab966c63c,USA


In [20]:
org_processing_df_v2['country_code'].value_counts()

USA    15277
GBR     1881
CHN     1461
DEU      955
CAN      884
       ...  
PRI        1
COM        1
TTO        1
AFG        1
KWT        1
Name: country_code, Length: 124, dtype: int64

In [21]:
top_10_countries = list(org_processing_df_v2['country_code'].value_counts().nlargest(10).index)

In [22]:
top_10_countries

['USA', 'GBR', 'CHN', 'DEU', 'CAN', 'FRA', 'IND', 'ESP', 'ISR', 'RUS']

In [23]:
country_code_modified = list()

for country_code in list(org_processing_df_v2['country_code']):
    if country_code in top_10_countries:
        country_code_modified.append(country_code)
    else:
        country_code_modified.append('others')

org_processing_df_v2['country_code_modified'] = country_code_modified

In [24]:
org_processing_df_v2['country_code_modified'].value_counts()

USA       15277
others     5431
GBR        1881
CHN        1461
DEU         955
CAN         884
FRA         756
IND         710
ESP         553
ISR         468
RUS         351
Name: country_code_modified, dtype: int64

In [25]:
#one hot encode for country

#create column for one-hot-encode for country
column_name = list()
for country_code in list(pd.get_dummies(org_processing_df_v2['country_code_modified']).columns):
    column_name.append(str('is_country_' + country_code.lower()))
    
country_one_hot = pd.get_dummies(org_processing_df_v2['country_code_modified'], columns=column_name)
new_column_dict = dict(zip(country_one_hot.columns,column_name))
country_one_hot.rename(columns = new_column_dict,inplace=True)

In [26]:
org_processing_df_v2 = pd.concat([org_processing_df_v2,country_one_hot],axis=1)

In [27]:
org_processing_df_v2.drop(columns = ['country_code','country_code_modified'],inplace=True)

In [28]:
org_processing_df_v2.head(3)

,org_uuid,is_country_can,is_country_chn,is_country_deu,is_country_esp,is_country_fra,is_country_gbr,is_country_ind,is_country_isr,is_country_rus,is_country_usa,is_country_others
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,0,0,0,0,0,0,0,0,0,1,0
1,9c8adac6-5c8e-9344-b763-6beab966c63c,0,0,0,0,0,0,0,0,0,1,0
2,6749cc07-efed-ff09-4efe-43faf6f580de,0,0,0,0,0,0,0,0,0,1,0


# Kauffman Entrepreneuship Indicator

In [29]:
org_processing_df_v3 = pd.merge(main_processing_df,
                                org_ori_df[['org_uuid','founded_on','country_code','state_code']],
                                on='org_uuid',
                                how='left')

In [30]:
org_processing_df_v3.head(3)

,org_uuid,founded_on,country_code,state_code
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,2000-05-01,USA,CA
1,9c8adac6-5c8e-9344-b763-6beab966c63c,2000-10-01,USA,MA
2,6749cc07-efed-ff09-4efe-43faf6f580de,2000-10-01,USA,CA


In [31]:
org_processing_df_v3['founded_year'] = org_processing_df_v3['founded_on'].dt.year

In [32]:
org_processing_df_v3.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,2000-05-01,USA,CA,2000
1,9c8adac6-5c8e-9344-b763-6beab966c63c,2000-10-01,USA,MA,2000
2,6749cc07-efed-ff09-4efe-43faf6f580de,2000-10-01,USA,CA,2000


In [33]:
kaufmann_index_df = pd.read_csv(R"d:\msc-project\data\kauffman\Kauffman_Indicators_Early-Stage_Entrepreneurship_Data_2020_v2.csv",encoding='utf-8')

In [34]:
kaufmann_index_df.head(3)

,name,region,type,category,year,rne,ose,sjc,ssr,zindex
0,Alabama,1,Total,Total,1996,NaN,NaN,6.131234,0.802180,NaN
1,Alabama,1,Total,Total,1997,NaN,NaN,6.507194,0.797268,NaN
2,Alabama,1,Total,Total,1998,0.0023,0.77638,5.954320,0.821527,-0.550553


In [35]:
#convert state name to abbreviation
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [36]:
kaufmann_index_df['state_code'] = kaufmann_index_df['name'].map(us_state_abbrev)

In [37]:
kaufmann_index_df.dropna(inplace=True)

In [38]:
kaufmann_index_df.head(3)

,name,region,type,category,year,rne,ose,sjc,ssr,zindex,state_code
2,Alabama,1,Total,Total,1998,0.00230,0.77638,5.954320,0.821527,-0.550553,AL
3,Alabama,1,Total,Total,1999,0.00206,0.82585,6.540875,0.814954,-0.339631,AL
4,Alabama,1,Total,Total,2000,0.00193,0.75509,4.982735,0.775368,-3.853653,AL


In [39]:
kaufmann_index_df['rne'] = (kaufmann_index_df['rne']*100).round(2)
kaufmann_index_df['ose'] = (kaufmann_index_df['ose']*100).round(2)
kaufmann_index_df['sjc'] = (kaufmann_index_df['sjc']).round(2)
kaufmann_index_df['ssr'] = (kaufmann_index_df['ssr']*100).round(2)
kaufmann_index_df['zindex'] = kaufmann_index_df['zindex'].round(2)

In [40]:
kaufmann_index_df.head(3)

,name,region,type,category,year,rne,ose,sjc,ssr,zindex,state_code
2,Alabama,1,Total,Total,1998,0.23,77.64,5.95,82.15,-0.55,AL
3,Alabama,1,Total,Total,1999,0.21,82.58,6.54,81.50,-0.34,AL
4,Alabama,1,Total,Total,2000,0.19,75.51,4.98,77.54,-3.85,AL


In [41]:
kaufmann_index_df = kaufmann_index_df[['state_code','year','rne','ose','sjc','ssr','zindex']]

In [42]:
kaufmann_index_df.head(3)

,state_code,year,rne,ose,sjc,ssr,zindex
2,AL,1998,0.23,77.64,5.95,82.15,-0.55
3,AL,1999,0.21,82.58,6.54,81.50,-0.34
4,AL,2000,0.19,75.51,4.98,77.54,-3.85


In [43]:
org_processing_df_v3.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,2000-05-01,USA,CA,2000
1,9c8adac6-5c8e-9344-b763-6beab966c63c,2000-10-01,USA,MA,2000
2,6749cc07-efed-ff09-4efe-43faf6f580de,2000-10-01,USA,CA,2000


In [44]:
org_processing_df_v3 = pd.merge(org_processing_df_v3,
                                kaufmann_index_df,
                                left_on=['state_code','founded_year'],
                                right_on=['state_code','year'],
                                how='left')

In [45]:
org_processing_df_v3.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year,year,rne,ose,sjc,ssr,zindex
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,2000-05-01,USA,CA,2000,2000.0,0.33,78.83,10.08,77.87,2.00
1,9c8adac6-5c8e-9344-b763-6beab966c63c,2000-10-01,USA,MA,2000,2000.0,0.16,80.54,7.39,82.46,-0.94
2,6749cc07-efed-ff09-4efe-43faf6f580de,2000-10-01,USA,CA,2000,2000.0,0.33,78.83,10.08,77.87,2.00


In [46]:
new_column_name_dict = {'rne':'kauffman_rate_of_new_entrepreneur',
                        'ose':'kauffman_opportunity_share',
                        'sjc':'kauffman_startup_early_job_creation',
                        'ssr':'kauffman_startup_early_survival_rate',
                       'zindex':'kauffman_zindex'}

In [47]:
org_processing_df_v3.rename(columns=new_column_name_dict,inplace=True)

In [48]:
org_processing_df_v3.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year,year,kauffman_rate_of_new_entrepreneur,kauffman_opportunity_share,kauffman_startup_early_job_creation,kauffman_startup_early_survival_rate,kauffman_zindex
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,2000-05-01,USA,CA,2000,2000.0,0.33,78.83,10.08,77.87,2.00
1,9c8adac6-5c8e-9344-b763-6beab966c63c,2000-10-01,USA,MA,2000,2000.0,0.16,80.54,7.39,82.46,-0.94
2,6749cc07-efed-ff09-4efe-43faf6f580de,2000-10-01,USA,CA,2000,2000.0,0.33,78.83,10.08,77.87,2.00


In [49]:
org_processing_df_v3.drop(columns=['founded_on','country_code','state_code','founded_year','year'],inplace=True)
org_processing_df_v3.head(3)

,org_uuid,kauffman_rate_of_new_entrepreneur,kauffman_opportunity_share,kauffman_startup_early_job_creation,kauffman_startup_early_survival_rate,kauffman_zindex
0,9dc17185-743d-5fbc-f03b-ad0c81c8795d,0.33,78.83,10.08,77.87,2.00
1,9c8adac6-5c8e-9344-b763-6beab966c63c,0.16,80.54,7.39,82.46,-0.94
2,6749cc07-efed-ff09-4efe-43faf6f580de,0.33,78.83,10.08,77.87,2.00


In [50]:
org_processing_df_v3.fillna(0,inplace=True)

In [51]:
len(org_processing_df_v3)

28727

# Rename columns and save final file

In [52]:
org_processing_df_v1.set_index('org_uuid', inplace=True)
org_processing_df_v2.set_index('org_uuid', inplace=True)
org_processing_df_v3.set_index('org_uuid', inplace=True)

In [53]:
main_processing_df = pd.concat([org_processing_df_v1,org_processing_df_v2,org_processing_df_v3], axis=1)

In [54]:
main_processing_df.head(3)

,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,is_consumer_electronics,...,is_country_ind,is_country_isr,is_country_rus,is_country_usa,is_country_others,kauffman_rate_of_new_entrepreneur,kauffman_opportunity_share,kauffman_startup_early_job_creation,kauffman_startup_early_survival_rate,kauffman_zindex
org_uuid,,,,,,,,,,,,,,,,,,,,,
9dc17185-743d-5fbc-f03b-ad0c81c8795d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0.33,78.83,10.08,77.87,2.00
9c8adac6-5c8e-9344-b763-6beab966c63c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0.16,80.54,7.39,82.46,-0.94
6749cc07-efed-ff09-4efe-43faf6f580de,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0.33,78.83,10.08,77.87,2.00


In [55]:
new_column_name = list()
for column in list(main_processing_df.columns):
    new_column_name.append('ecosystem_'+ column)

In [56]:
new_column_dict = dict(zip(main_processing_df.columns,new_column_name))
main_processing_df.rename(columns = new_column_dict,inplace=True)

In [57]:
main_processing_df.head(3)

,ecosystem_is_administrative_services,ecosystem_is_advertising,ecosystem_is_agriculture_and_farming,ecosystem_is_apps,ecosystem_is_artificial_intelligence,ecosystem_is_biotechnology,ecosystem_is_clothing_and_apparel,ecosystem_is_commerce_and_shopping,ecosystem_is_community_and_lifestyle,ecosystem_is_consumer_electronics,...,ecosystem_is_country_ind,ecosystem_is_country_isr,ecosystem_is_country_rus,ecosystem_is_country_usa,ecosystem_is_country_others,ecosystem_kauffman_rate_of_new_entrepreneur,ecosystem_kauffman_opportunity_share,ecosystem_kauffman_startup_early_job_creation,ecosystem_kauffman_startup_early_survival_rate,ecosystem_kauffman_zindex
org_uuid,,,,,,,,,,,,,,,,,,,,,
9dc17185-743d-5fbc-f03b-ad0c81c8795d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0.33,78.83,10.08,77.87,2.00
9c8adac6-5c8e-9344-b763-6beab966c63c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0.16,80.54,7.39,82.46,-0.94
6749cc07-efed-ff09-4efe-43faf6f580de,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0.33,78.83,10.08,77.87,2.00


In [59]:
for column in list(main_processing_df.columns):
    print(column)

ecosystem_is_administrative_services
ecosystem_is_advertising
ecosystem_is_agriculture_and_farming
ecosystem_is_apps
ecosystem_is_artificial_intelligence
ecosystem_is_biotechnology
ecosystem_is_clothing_and_apparel
ecosystem_is_commerce_and_shopping
ecosystem_is_community_and_lifestyle
ecosystem_is_consumer_electronics
ecosystem_is_consumer_goods
ecosystem_is_content_and_publishing
ecosystem_is_data_and_analytics
ecosystem_is_design
ecosystem_is_education
ecosystem_is_energy
ecosystem_is_events
ecosystem_is_financial_services
ecosystem_is_food_and_beverage
ecosystem_is_gaming
ecosystem_is_government_and_military
ecosystem_is_hardware
ecosystem_is_health_care
ecosystem_is_information_technology
ecosystem_is_internet_services
ecosystem_is_lending_and_investments
ecosystem_is_manufacturing
ecosystem_is_media_and_entertainment
ecosystem_is_messaging_and_telecommunications
ecosystem_is_mobile
ecosystem_is_music_and_audio
ecosystem_is_natural_resources
ecosystem_is_navigation_and_mapping
eco

In [63]:
len(main_processing_df.columns)

63

In [58]:
main_processing_df.to_csv(r'd:\msc-project\data\final\features_ecosystem.csv',encoding='utf=8')